<a href="https://colab.research.google.com/github/valenlopez993/SMS_Classifier/blob/main/SMS_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
  !pip install tensorflow-datasets
except Exception:
  pass

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.utils import pad_sequences

from tensorflow.keras.preprocessing.text import Tokenizer

import tensorflow_datasets as tfds

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

print(tf.__version__)

# Getting the Data

In [ ]:
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
df_train = pd.read_csv(train_file_path, sep='\t', header=None, names=['ham_spam', 'mail'])
df_test = pd.read_csv(test_file_path, sep='\t', header=None, names=['ham_spam', 'mail'])

In [ ]:
df_train_labels = df_train.pop('ham_spam')
df_test_labels = df_test.pop('ham_spam')

df_train_labels = pd.Categorical(df_train_labels)
df_test_labels = pd.Categorical(df_test_labels)

train_labels = df_train_labels.codes
test_labels = df_test_labels.codes

categories = df_train_labels.unique()

In [ ]:
train_to_tokenize = df_train.to_numpy().reshape(df_train.shape[0])
train_tokenizer = Tokenizer()
train_tokenizer.fit_on_texts(train_to_tokenize)
train_tokenized = train_tokenizer.texts_to_sequences(train_to_tokenize)

test_to_tokenize = df_test.to_numpy().reshape(df_test.shape[0])
test_tokenizer = Tokenizer()
test_tokenizer.fit_on_texts(test_to_tokenize)
test_tokenized = test_tokenizer.texts_to_sequences(test_to_tokenize)

In [ ]:
word_index = train_tokenizer.word_index
vocab_size = len(word_index) + 1

In [ ]:
maxLength = 255
train_data = pad_sequences(train_tokenized, maxLength)
test_data = pad_sequences(test_tokenized, maxLength)

# Model

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 32))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit(train_data, train_labels, epochs=50)

# Final Test

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts([pred_text])
  tokenized = train_tokenizer.texts_to_sequences([pred_text])
  text_to_predict = pad_sequences(tokenized, maxLength)

  model_prediction = model.predict(text_to_predict)
  
  prediction = []
  prediction.append(model_prediction[0][0])
  prediction.append(categories[int(np.round(model_prediction)[0])])

  return prediction

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
